# OpenQASM Parser Library

This notebook is intended to provide an insight into the _OpenQASM Parser Library_, able to parse a common quantum circuit description format used in the context of variational algorithms into the corresponding OpenQASM representation, utilizing parallel computing. 

## Prerequisites

To be able to use the library additional libraries must be installed. These include:
- OpenMP
- Thread Building Blocks (TBB)
- fmt

Further, we are using [*cmake*](https://cmake.org/download/) (already installed on many systems or part of the package manager) as our generator and GCC as our compiler. If not yet the default C/C++ compiler, install the *GNU Compiler Collection* (GCC) as your compiler to use the library. Some older versions of the GCC may must be upgraded to be used with the library. The reason for that is the necessity to use the execution policies first implemented in the C++17 standard. The GCC can be an globally existing binary, but other than that I would highly emphasize using a virtual environment, since we will install the library as an C++ Python binding and one probably does not want to install everything globally and eventually mess with the system on a global scale. An easy to use and project compatible virtual environment manager is [*Anaconda*](https://docs.anaconda.com/free/anaconda/install/index.html). Install Anaconda on your system and [create a virtual environment](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html) with a fairly new Python version.

Assuming the used environment manager is Anaconda, installing the additional necessary libraries can be done like:
```
conda install -c conda-forge openmp
conda install -c conda-forge tbb
conda install -c conda-forge fmt
```

## Adding the Library to the Python Packages

After cloning the project in an appropriate location, we need to build and install the library. Therefore, the location of your Python *site-packages* folder must be known. The path can be found using `python -m site` command in a shell/terminal session. Be aware that the python binary pointed to is the correct one, meaning the one you want to be using. Verify by first executing `which python` and confirm that the path leads to your desired binary. In a virtual environment this should point to a binary in your virtual environment folder, e.g. `/Users/maxmustermann/opt/anaconda3/envs/environment_name/bin/python3.10` in Unix-like systems. The site packages should also be found in a similar location, e.g. `/Users/maxmustermann/opt/anaconda3/envs/environment_name/lib/python3.10/site-packages`.

After that, we are ready to build and install. In the library directory created when cloning the library from GitHub, we create a directory for our build. After changing into that directory we build the library and finally install it. For Unix-like systems the commands should be the following:
```
mkdir build && cd build
cmake .. -DCMAKE_C_COMPILER=/path_or_alias/to/your/gcc-binary -DCMAKE_CXX_COMPILER=/path_or_alias/to/your/g++-binary -DPYTHON_LIBRARY_DIR="<your_path_to_your_site-packages>" -DPYTHON_EXECUTABLE="<your_path_to_your_python_binary>"
cmake --build .
cmake --install .
```
(The compiler must only be provided if the GCC is not yet the default compiler. Quotation marks are necessary!)

The library should now be installed as a python package. You can confirm by changing into the python site-packages directory and reviewing the content. A shared *openqasmparser* library should now be visible. You could also fire up a python session and try to import the library 
```
python
> import openqasmparser as qprs
>
```
If no error message pops up you are ***ready to go!***

In [1]:
import openqasmparser as qprs

## OpenQASM

_Open Quantum Assembly Language_ (OpenQASM) is considered to be the de facto standard when it comes to communicating with a quantum computing device. It builds a bridge from describing the circuit on classical hardware to actually implementing it on quantum hardware. Such representation, residing in between the classical and quantum part of the workflow, are called _intermediate representations_. The sole purpose of these intermediate representations is to be able to easily construct quantum circuits and be able to implement those on various different quantum computing devices with (mostly) no additional cost for the user. 

Currently, two major OpenQASM versions are available, [version 2](https://arxiv.org/abs/1707.03429) and [3](https://arxiv.org/abs/2104.14722). Version 3 further extends the capabilities of the intermediate representation and provides the user with more advanced possibilities to construct circuits and instruct quantum devices. As they describe it: [In principle, any quantum computation could be described using OpenQASM 2, but there is a need to describe a broader set of circuits beyond the language of qubits and gates.][1] The library presented here is indifferent between the two versions since the more advanced instructions working with the circuit described/generated are not (yet) implemented, neither in the library implementation itself nor in the input representation. Nevertheless, both versions are supported, the only difference is in the headers specific to the specified version used. Both headers can be seen in the following:

[1]: https://arxiv.org/abs/2104.14722

In [2]:
header_v2 = qprs.parse_circuit("", version=2)
header_v3 = qprs.parse_circuit("", version=3)

print(header_v2)
print(header_v3)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[6];
creg c[6];

OPENQASM 3.0;
include "stdgates.inc";
qubit[6] q;
bit[6] c;



## Input Representation

The goal of the library is to minimize the effort needed to instruct quantum computing devices. The instruction can happen directly, on quantum hardware level, or using the intermediate representations, e.g. OpenQASM instruction files. Instructing quantum devices manually is too tedious and time consuming for the upcoming larger scale quantum devices and poses a threat for a bottleneck in the quantum computing workflow. Intermediate representations can be constructed once and used on different devices without the need of knowing the exact hardware capabilities of the devices at hand. The library's input representation used could possibly be an intermediate representation by itself, but considering the broad usage and support of OpenQASM using OpenQASM as an intermediate representation simplifies and generalizes the usage on most devices. 

OpenQASM is a code-like language with which one could write his desired circuits by hand. But even these files will become of such large scale for problems suitable on future more capable devices that writing these by hand will also reach its feasible limits. Therefore, we propose a different representation, easy to generate and well suited for the usage in the variational quantum algorithms context. 

The Pauli operations form together with the identity a complete basis set, meaning we can construct any arbitrary operation using these four by suitable transformation. They also have an added advantage, since they are such basic operations nearly every quantum computing device has the hardware capabilities to perform these operations directly. The input file to generate can be a simple text file. The Pauli operations are represented by the capital letters *X, Y, Z* and the identity as the capital letter *I*. We now represent the ansatz, a (parameterized) circuit, as an unitary collection of *m* operators which is basically a product of *m* sequentially applied unitaries:

$$ \mathbb{U}(\boldsymbol{\theta}) = \mathbb{U}_m(\theta_m) \dots \mathbb{U}_2(\theta_2) \mathbb{U}_1(\theta_1)$$

Therfore, our input file consists of *m* lines, one for each operator unitary of the ansatz. Each of the unitaries is composed of multiple gates, parameterized and unparameterized. Looking at a single unitary acting on a system with *N* qubits in its exponentiated form

$$ \mathbb{U}_i(\theta_i) = \exp\{-i \theta_i \hat{a}_i^{\otimes N}\} $$
where 
$$ \hat{a}_i^{\otimes N} = \otimes_{l=1}^N \Theta_{i,l}\ :\ \Theta \in \{X,Y,Z,I\}$$
we find the Pauli and identity operations in \\(\hat{a}\\). This tensor product will build the foundation of our input representation, where the tensor product of Paulis and identity is written as a string \\(S_i\\). Consider the unitary \\(\exp{-i \theta \sigma_x \otimes \sigma_I \otimes \sigma_x}\\), the string representation as part of the input file to the library would simply be *XIX*. Another variable to set in the input file is the coefficient \\(c_i\\), seperated from the string by whitespace and in floating point accuracy, corresponding to the real-valued parameter \\(\theta_i\\). The last parameter of our input representation if not to be seen in the mathematical description of the operator. In order to group commuting operators together, reducing the computational effort when optimizing the parameters, we introduce an enumerating grouping index \\(p_i\\). The same index indicates commuting operators, while different grouping index parameters indicate non-commuting operators which must be optimized individually. Putting everything together we have our input representation to generate:

$$ A \equiv \{(S_i, c_i, p_i)\ :\ i \in \{1, \dots m\}\} $$

To get a better understanding let's look at an example. Consider an ansatz for \\(N=3\\) qubits and \\(m=2\\) independent operators as 

$$ \exp\{-i \cdot 1.0 \cdot \sigma_y \otimes \sigma_y \otimes \sigma_I\} \exp\{-i \cdot 1.5 \cdot \sigma_y \otimes \sigma_I \otimes \sigma_z\} $$

The corresponding input file to our library would look as follows:
> YYI 1.0 1
> 
> YIZ 1.5 2

Note how the grouping index enumerates, indicating independent operators. The same could be achieved by setting both grouping index parameters to 0, the algorithm then enumerates independent parameters by itself. The same ansatz assuming commuting operators which can be grouped would look similar
> YYI 1.0 1
>
> YIZ 1.5 1

but with the same grouping index to indicate the dependency.



## Using the Library

Now that we know how to construct an input file to be parsed into the OpenQASM representation, let's see how the library actually works. Therefore, take a look at the following examplaratory input ansatz. The ansatz is already in our library's desired input format and part of an actual ansatz used in the variational context on today's quantum computing devices. The ansatz can also be found in the additional data provided. 

In [3]:
import pandas as pd

ex_input = pd.read_csv("h2short.ansatz", sep=" ", header=None, names=["S_i", "c_i", "p_i"])
print(ex_input.to_string(index=False))


         S_i  c_i  p_i
IIIIIIIIIIXX  1.0    1
IIIIIIIIIIXY  1.0    2
IIIIIIIIIIXZ  1.0    3
IIIIIIIIIIYX  1.0    4
IIIIIIIIIIYY  1.0    5
IIIIIIIIIIYZ  1.0    6
IIIIIIIIIIZX  1.0    7
IIIIIIIIIIZY  1.0    8
IIIIIIIIIIZZ  1.0    9
IIIIIIIIIXIX  1.0   10


This is quite the compact format compared to other representations, as we will see soon. We can immediatly see two things when looking at the input file. First, the operators are all independent, which can be seen by the enumerating grouping index parameter \\(p_i\\). And second, this input file is actually unparameterized. All coefficients are set to 1, who's multiplication has no observable effect. 

To parse the input file in its OpenQASM representation, the library provides a function *parse_circuit*. Let's take a look at the function's help manual to see what we can do with it and how to call it.

In [4]:
help(qprs.parse_circuit)

Help on built-in function parse_circuit in module openqasmparser:

parse_circuit(...) method of builtins.PyCapsule instance
    parse_circuit(input_fn: str, *, version: int = 3, use_omp: bool = False, parameterize: bool = True, output_fn: Optional[str] = None, multiplier: Optional[float] = None) -> str
    
    Parse an ansatz into the corresponding OpenQASM representation, parallel execution enabled.
    @param input_fn: Path to the input file to parse.
    @param version: OpenQASM version to use, default 2.
    @param use_omp: Use OpenMP parallelism, default execution policy parallelism.
    @param parameterize: Set true to use indication for grouping commuting operators.
    @param output_fn: Path to (non-)existing file to store the OpenQASMrepresentation file. (optional)
    @param multiplier: Floating value to be multiplied to each operator. (optional)



We can specify five different variables. The only variable that must be set is the path to the input file. This variable is positional, meaning we need to specify this variable first when calling the function. All other variables are key-word only. In order to specify them you have to set the variable at call in the standard pythonic way, e.g. *use_omp=True*.

- *version*
  The version can be set to be OpenQASM v2 or v3. The only real difference is the header specific to that version in the output OpenQASM file. If not provided or set to a value other than 2 or 3, the default version used is version 3. To use parameterization version 3 is necessary.

- *use_omp*
  If parallelization is supported by your system, it is enabled by default. One can, for scientific/benchmarking purposes for example, decide which parallelization framework to use. If set to *True* OpenMP is the underlying parallelization framework, otherwise and by default the C++17 execution policies are used. These have the added advantage of automatic compiler optimization, leaving less room for unnecassary occupancy of resources or errors in general. If one wishes, he can tailor the OpenMP implementation in the C++ library source code to his specific needs.

- *parameterize*
  To indicate commuting operators we use the grouping index. This index is not visible in the OpenQASM representation directly, but indicates the usage of the same parameter for the rotation in z-basis. If one does not want to parameterize anyways, disabling the parameterization can be controlled using this variable.

- *output_fn*
  If specified, the OpenQASM representation is written to the file in that location. This value is optional. Nevertheless, the output of the function is still accessible in the current session, e.g. stored in a variable bound to that call.

- *multiplier*
  Value to be multiplied to each operator. If not set specifically, the circuit implementation uses a multiplier of \\(2\\). By changing this variable one can change the magnitude of the parameterization.

Now, calling the function and displaying the output looks like this:

In [5]:
# You can try out different additional parameters
# We parse the h2short.ansatz ansatz and store it in our calling directory of the notebook
circ1 = qprs.parse_circuit("h2short.ansatz", version=3, output_fn="./h2short.qasm", parameterize=True)
print(circ1)

OPENQASM 3.0;
include "stdgates.inc";
qubit[12] q;
bit[12] c;
input float param1;
input float param2;
input float param3;
input float param4;
input float param5;
input float param6;
input float param7;
input float param8;
input float param9;
input float param10;

// New operator from line 1
ry(pi/2) q[11];
ry(pi/2) q[10];
cx q[10], q[11];
rz(2*1*param1) q[11];
cx q[10], q[11];
ry(-pi/2) q[10];
ry(-pi/2) q[11];

// New operator from line 2
rx(-pi/2) q[11];
ry(pi/2) q[10];
cx q[10], q[11];
rz(2*1*param2) q[11];
cx q[10], q[11];
ry(-pi/2) q[10];
rx(pi/2) q[11];

// New operator from line 3
ry(pi/2) q[10];
cx q[10], q[11];
rz(2*1*param3) q[11];
cx q[10], q[11];
ry(-pi/2) q[10];

// New operator from line 4
ry(pi/2) q[11];
rx(-pi/2) q[10];
cx q[10], q[11];
rz(2*1*param4) q[11];
cx q[10], q[11];
rx(pi/2) q[10];
ry(-pi/2) q[11];

// New operator from line 5
rx(-pi/2) q[11];
rx(-pi/2) q[10];
cx q[10], q[11];
rz(2*1*param5) q[11];
cx q[10], q[11];
rx(pi/2) q[10];
rx(pi/2) q[11];

// New operato

A lot of lines of code-like OpenQASM, considering the input file consisted of only ten operators working on twelve qubits. Imagine the OpenQASM files of ansätze consisting of thousands of operators. You can try yourself setting other function call parameters.

Some remarks regarding the output. One can see the parameter initialization in the first lines after the header. The numbering actually corresponds to the grouping index. Trying to run this OpenQASM file on a simulator for example would result in an error. To do so, we first have to bind the parameters to the specific \\(\theta\\) values. Commuting operators use the same parameter and reduces the computational effort when optimizing since lesser parameters need to be optimized. Other than that, we can also see two numbers being multiplied before that. The second number is the coefficient of the input file. The other number can be set using the multiplier and if not set explicitly is \\(2.0\\). We explain that in the following, as well as why there are so many rotations.

But first, let's look at an actual circuit `test.ansatz`. We can use qiskits functionality to load circuits from OpenQASM files or strings (Additional qiskit modules may be required). I am using the `qasm3` module for v3 and the [`qiskit_qasm2` library](https://pypi.org/project/qiskit-qasm2/) for v2 and their respective `load` (for files) and `loads` (for strings) functionality. Note: To use visualization and/or jupyter notebooks the [visualization extra requirements](https://qiskit.org/documentation/stable/0.24/install.html) are recommended.

In [52]:
from qiskit import qasm3, BasicAer, transpile
import qiskit_qasm2

ansatz = qprs.parse_circuit("test.ansatz", parameterize=True, version=3)
qc = qasm3.loads(ansatz)
qc.draw('mpl')

In [54]:
backend = BasicAer.get_backend('qasm_simulator')

# Since there are no measurements, let's add them
qc.measure_all(add_bits=False)

# Bind the parameters by using an iterable (dict, list...)
pqc = qc.assign_parameters([1.57])

# Transpile circuit to fit used backend
tqc = transpile(pqc, backend)

# Run circuit and get measurement
counts = backend.run(tqc).result().get_counts()

print(counts)

{'11': 1024}


We can see that we obtain the state \\(\ket{11}\\) in all runs. In the next chapter we see the mathematical desciption of the operator's circuit implementation and one can calculate the result on paper. When using \\(\theta = \frac{\pi}{2}\\) the amplitude of the state \\(\ket{11}\\) corresponds to \\(1\\) and therefore the probability to obtain this state is \\(|1|^2 = 1\\).

This way we can use the library in a qiskit workflow for example to simplify the circuit generation. We used two commuting operators, sharing the same parameter. Further, we set the parameter to \\(\frac{\pi}{2}\\). You can try yourself changing the parameter(s) and/or ansatz. For example, what happens changing the parameter to \\(\pi\\), using non-commuting operators and specifying two parameters, ...

## Circuit Implementation

Imagine the simple input *ZZ*, who's exponentiation is given by \\(\exp\{-i \theta \sigma_z \otimes \sigma_z\}\\). To implement this operator in a circuit we can use *CNOT* and the rotation gate along the z-axis, simply sandwiching the rotation between *CNOT* gates. It is best understood when looking at the matrix form of the gates and operator:

$$ \exp{-i\theta \sigma_z \otimes \sigma_z} = \cos\left(\theta\right) \cdot I - i \sin\left(\theta\right) \sigma_z \otimes \sigma_z = 
\begin{pmatrix}
e^{-i\theta} & 0 & 0 & 0 \\
0 & e^{i\theta} & 0 & 0 \\
0 & 0 & e^{i\theta} & 0 \\
0 & 0 & 0 & e^{-i\theta} 
\end{pmatrix} $$

which is the matrix form of the operator, obtained by using Euler's formula, we are trying to implement in a circuit. Now, looking at the sandwiched rotation we obtain the same operator

$$ \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 
\end{pmatrix}
\begin{pmatrix}
e^{-i\theta} & 0 & 0 & 0 \\
0 & e^{i\theta} & 0 & 0 \\
0 & 0 & e^{-i\theta} & 0 \\
0 & 0 & 0 & e^{i\theta} 
\end{pmatrix}
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 
\end{pmatrix} = 
\begin{pmatrix}
e^{-i\theta} & 0 & 0 & 0 \\
0 & e^{i\theta} & 0 & 0 \\
0 & 0 & e^{i\theta} & 0 \\
0 & 0 & 0 & e^{-i\theta} 
\end{pmatrix}
$$

This is the most basic example, since we are only working in our computational basis, the z-axis. If other Pauli operations are present in the operator we need to rotate beforehand to actually perform the operator in the right computational basis. For example imagine another simple operator *XX*, doing the same comparison as before, we want to obtain following operator in matrix form

$$e^{-i\theta \sigma_x \otimes \sigma_x} = 
\begin{pmatrix}
\cos(\theta) & 0 & 0 & -i \sin(\theta) \\
0 & \cos(\theta) & -i \sin(\theta) & 0 \\
0 & -i \sin(\theta) & \cos(\theta) & 0 \\
-i \sin(\theta) & 0 & 0 & \cos(\theta)
\end{pmatrix} \\
$$

and therefore we use parameterized rotation along the y-axis by \\(\frac{\pi}{2}\\) counterclockwise before and clockwise after the parameterized rotation in z-basis sandwiched by *CNOT* gates, given by the matrices

$$R_Y(\frac{\pi}{2}) \otimes R_Y(\frac{\pi}{2}) = 
\begin{pmatrix}
\frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} \\
\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\
\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\
\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} 
\end{pmatrix} \\ \quad ; \quad
R_Y(-\frac{\pi}{2}) \otimes R_Y(-\frac{\pi}{2}) = 
\begin{pmatrix}
\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\
-\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} \\
-\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\
\frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} 
\end{pmatrix}
$$

We use the matrix result for the sandwiched rotation along the z-axis between *CNOT* gates to obtain

$$
\begin{pmatrix}
\frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} \\
\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\
\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\
\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} 
\end{pmatrix} 
\begin{pmatrix}
e^{-i\theta} & 0 & 0 & 0 \\
0 & e^{i\theta} & 0 & 0 \\
0 & 0 & e^{i\theta} & 0 \\
0 & 0 & 0 & e^{-i\theta} 
\end{pmatrix}
\begin{pmatrix}
\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\
-\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} \\
-\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\
\frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} 
\end{pmatrix} \\
= 
\begin{pmatrix}
\cos(\theta) & 0 & 0 & -i \sin(\theta) \\
0 & \cos(\theta) & -i \sin(\theta) & 0 \\
0 & -i \sin(\theta) & \cos(\theta) & 0 \\
-i \sin(\theta) & 0 & 0 & \cos(\theta)
\end{pmatrix} \\
$$

and we can see that indeed the operator is the same. This actually means, that the universal rotation gate or the rotation gates along the axis and the *CNOT* present in the hardware implementation of the quantum device used are sufficient. State of the art quantum computers may even optimize the circuit to their specific hardware capabilities, but this form is the most general and applicable using the fewest and most basic gates.

For the Pauli-Y operation we perform the rotation along the x-axis clockwise before and counterclockwise afterward. Feel free to calculate the *YY* operator on your own to see that the result is indeed the same. 

This way, we have shown that all arbitrary combinations of Pauli operations in an operator can be implemented in a quantum circuit using rotation and *CNOT* gates, just like the library does.

Furthermore, looking at the parameterized rotation along the z-axis we see that the rotation is actually parameterized by \\(2\theta\\). This is the reason for the multiplier to be set to \\(2.0\\) by default. Other values are also possible and can be set by the multiplier parameter in the library function. One has to be aware of what he is actually trying to achieve by doing so.